In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# Load the DataFrame
df = pd.read_csv("soilmoisture_dataset.csv", index_col=0)

# Remove empty rows
df = df.dropna()

# Reset the index without adding the old index as a column
df.reset_index(drop=True, inplace=True)

# Drop unnecessary columns
df = df.drop(['soil_temperature', 'datetime'], axis=1)

# Split 90% of the data into `train_val` and 10% into `test`
train_val, test = train_test_split(df, test_size=0.2, random_state=42)

X_cross_val = train_val.drop('soil_moisture', axis=1)
y_cross_val = train_val['soil_moisture']

X_test2 = test.drop('soil_moisture', axis=1)
y_test2 = test['soil_moisture']

In [16]:
import pandas as pd
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from math import sqrt

# Define the adjusted R-squared function
def adjR2(r2, n_features, n_samples):
    return 1 - (1 - r2) * (n_samples - 1) / (n_samples - n_features - 1)

# Create a LinearSVR model
model = LinearSVR(C=250, random_state=42, dual="auto", max_iter=10000)
model2 = SVR(kernel='rbf', C=1000, gamma='scale')

k = 15  # number of total features to compare

# Initialize a list to store results
results = []

for n in range(k):
    # Create the RFE selector with the model
    rfe = RFE(model, n_features_to_select=n + 1)

    # Fit the RFE selector on the data
    fit = rfe.fit(X_cross_val, y_cross_val)

    # Get the selected features as a boolean array
    selected_features = fit.support_

    # Create a list of selected feature names
    selected_columns = X_cross_val.columns[selected_features].tolist()

    print(f"Selected columns (iteration {n + 1}):", selected_columns)

    # Ensure all selected columns exist in data
    missing_columns = [col for col in selected_columns if col not in X_cross_val.columns]
    if missing_columns:
        print(f"Missing columns in data for iteration {n + 1}: {missing_columns}")
        continue  # Skip iteration if columns are missing

    # Perform 5-fold cross-validation
    kf = KFold(n_splits=10, shuffle=True, random_state=42)

    rmse_list = []
    adjR_list = []

    for train_index, test_index in kf.split(X_cross_val):
        # Split the data into training and testing sets
        X_train, X_test = X_cross_val.iloc[train_index][selected_columns], X_cross_val.iloc[test_index][selected_columns]
        y_train, y_test = y_cross_val.iloc[train_index], y_cross_val.iloc[test_index]

        # Scale the features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)  # Fit and transform on training data
        X_test_scaled = scaler.transform(X_test)  # Transform the test data

        # Train the model on the scaled data
        model2.fit(X_train_scaled, y_train)
        y_pred = model2.predict(X_test_scaled)

        # Calculating mean squared error
        mse = mean_squared_error(y_test, y_pred)
        rmse_list.append(sqrt(mse))

        # Calculating adjusted R squared
        r2 = r2_score(y_test, y_pred)
        adjR_list.append(adjR2(r2, X_test.shape[1], X_test.shape[0]))

    # Average RMSE across folds
    average_rmse = sum(rmse_list) / len(rmse_list)
    average_adjR = sum(adjR_list) / len(adjR_list)

    # Print results
    print("Average RMSE:", round(average_rmse, 4), "Average Adjusted R-squared:", round(average_adjR, 4))

    # Append the results to the list
    results.append({
        'Iteration': n + 1,
        'Selected Columns': ', '.join(selected_columns),  # Store the selected columns as a string
        'Average RMSE': round(average_rmse, 4),
        'Average Adjusted R²': round(average_adjR, 4)
    })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Save results to Excel
results_df.to_excel("RFE_results.xlsx", index=False)

print("Results saved to 'RFE_results.xlsx'")


Selected columns (iteration 1): ['950']
Average RMSE: 1.9076 Average Adjusted R-squared: 0.7019
Selected columns (iteration 2): ['494', '950']
Average RMSE: 1.4631 Average Adjusted R-squared: 0.8128
Selected columns (iteration 3): ['462', '494', '950']
Average RMSE: 1.325 Average Adjusted R-squared: 0.8439
Selected columns (iteration 4): ['462', '494', '498', '950']
Average RMSE: 1.3271 Average Adjusted R-squared: 0.8408
Selected columns (iteration 5): ['462', '494', '498', '766', '950']
Average RMSE: 1.1206 Average Adjusted R-squared: 0.8837
Selected columns (iteration 6): ['462', '494', '498', '506', '766', '950']
Average RMSE: 1.1228 Average Adjusted R-squared: 0.8818
Selected columns (iteration 7): ['462', '490', '494', '498', '506', '766', '950']
Average RMSE: 1.1316 Average Adjusted R-squared: 0.8777
Selected columns (iteration 8): ['462', '490', '494', '498', '506', '762', '766', '950']
Average RMSE: 1.1273 Average Adjusted R-squared: 0.8759
Selected columns (iteration 9): ['462